## KorQuad 와 비슷한 형태의 nested arrow 를 pandas로 불러와서 전처리하는 notebook
nested arrow를 pandas로 불러오고, 다시 nested arrow로 돌려놓는 과정이 어려웠기에 정리했습니다.  
KorQuad dataset, 대회 dataset은 다행히도 위키만큼 지저분하진 않아서 newline만 삭제해주는 전처리를 했습니다.  
그러나 `\n`과 `\\n` 이 혼용되어 있음에 주의할 필요가 있습니다.

In [6]:
import pandas as pd
import pyarrow as pa
from datasets import load_from_disk

In [36]:
train_data = load_from_disk('combined')['train']
train_data

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 70133
})

nested가 아닌 부분만 먼저 불러다가 DataFrame으로 바꿔줍니다

In [37]:
df = pd.DataFrame()
title, context, question, id, answers, document_id = train_data['title'],train_data['context'],train_data['question'], train_data['id'],train_data['answers'],train_data['document_id']
df.insert(0,'title',title)
df.insert(1,'context',context)
df.insert(2, 'question', question)
df.insert(3, 'id', id)
# df.insert(4, 'answer_start', answers['answer_start'][0])
# df.insert(5, 'answer_text', answers['text'][0])
df.insert(4, 'document_id', document_id)

df

,title,context,question,id,document_id
0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,6566495-0-0,-1
1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,6566495-0-1,-1
2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,6566495-0-2,-1
3,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,6566518-0-0,-1
4,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,6566518-0-1,-1
...,...,...,...,...,...
70128,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,8275
70129,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,37764
70130,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,12730
70131,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,10924


여기가 헤멘 nested 부분입니다. 
데이터셋 구조상 `answers` 안에 `text`와 `answer_start` 가 들어있어서 전에는 for문을 돌면서 일일이 뽑아왔었습니다.  
3000개짜리 대회 데이터셋에서는 참을만 하지만 70000개가 되니? 도저히 못 참겠더군요.   
의외로 `train_data[answers]` 를 DataFrame으로 불러오니 nested가 해제되고 dataframe으로 금방 바꿀 수 있었습니다.

In [38]:
ans = train_data['answers']
answer = pd.DataFrame(ans)
answer

,text,answer_start
0,[교향곡],[54]
1,[1악장],[421]
2,[베토벤의 교향곡 9번],[194]
3,[파우스트],[15]
4,[합창교향곡],[354]
...,...,...
70128,[지질 구조],[317]
70129,[뉴저지],[616]
70130,[칼 에케르트],[255]
70131,[슬라브 신화],[476]


In [39]:
train_data['answers'][0]['text']
#type(train_data['answers'][0]['text'])

['교향곡']

예시로 하나를 찍어봤더니 list형태 그대로 [] 가 떨어지지 않고 string이 되어 dataframe에 들어간 게 보이네요. 앞뒤를 잘라주는 함수를 만들겠습니다

In [ ]:
cut_punct()

In [40]:
def cut_punct(inp):
    print(inp)
    return inp[1:-1]

In [41]:
answer['text'] = answer['text'].apply(lambda x: x[0])
answer['answer_start'] = answer['answer_start'].apply(lambda x: x[0])
answer

,text,answer_start
0,교향곡,54
1,1악장,421
2,베토벤의 교향곡 9번,194
3,파우스트,15
4,합창교향곡,354
...,...,...
70128,지질 구조,317
70129,뉴저지,616
70130,칼 에케르트,255
70131,슬라브 신화,476


보기 싫은 [] 가 없어졌군요. 이제 두 dataframe을 하나로 합치겠습니다.

In [42]:
df = pd.concat([df,answer], axis = 1)
df

,title,context,question,id,document_id,text,answer_start
0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,6566495-0-0,-1,교향곡,54
1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,6566495-0-1,-1,1악장,421
2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,6566495-0-2,-1,베토벤의 교향곡 9번,194
3,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,6566518-0-0,-1,파우스트,15
4,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,6566518-0-1,-1,합창교향곡,354
...,...,...,...,...,...,...,...
70128,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,8275,지질 구조,317
70129,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,37764,뉴저지,616
70130,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,12730,칼 에케르트,255
70131,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,10924,슬라브 신화,476


첫번째로 `\n` 을 지웠을 때 새로 할당해야 하는 answer_start index를 변경하는 함수입니다.

In [43]:
def new_index(sentence,answer_start):
    minus = sentence[:answer_start].count('\\n')+(sentence[:answer_start].count('\n'))*2
    return answer_start-minus

In [44]:
df['answer_start'] = df.apply(lambda row:new_index(row['context'],row['answer_start']), axis = 1)
df

,title,context,question,id,document_id,text,answer_start
0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,6566495-0-0,-1,교향곡,54
1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,6566495-0-1,-1,1악장,421
2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,6566495-0-2,-1,베토벤의 교향곡 9번,194
3,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,6566518-0-0,-1,파우스트,15
4,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,6566518-0-1,-1,합창교향곡,354
...,...,...,...,...,...,...,...
70128,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,8275,지질 구조,313
70129,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,37764,뉴저지,610
70130,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,12730,칼 에케르트,253
70131,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,10924,슬라브 신화,468


두번째로 context에서 `\n` 을 지우는 함수입니다. 

In [26]:
def del_nl(sentence):
    sentence=sentence.replace('\n','')
    sentence=sentence.replace('\\n','')
    return sentence

In [27]:
df['context'] = df['context'].apply(lambda x : del_nl(x))
df

,title,context,question,id,document_id,text,answer_start
0,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?,6566495-0-0,-1,교향곡,54
1,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?,6566495-0-1,-1,1악장,421
2,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?,6566495-0-2,-1,베토벤의 교향곡 9번,194
3,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?,6566518-0-0,-1,파우스트,15
4,파우스트_서곡,1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로...,파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?,6566518-0-1,-1,합창교향곡,354
...,...,...,...,...,...,...,...
70128,이오 (위성),이오의 산이오의 산 목록 이오에는 100~150개의 산이 있다. 이들 산의 평균 높...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,8275,지질 구조,309
70129,내니 다이어리,애니의 고군분투 뉴욕 입성기!!인류학자가 꿈인 21살 소녀 '애니(스칼렛 요한슨 분...,애니는 어디서 태어났는가?,mrc-1-000699,37764,뉴저지,604
70130,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,12730,칼 에케르트,251
70131,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,10924,슬라브 신화,460


`\n` 이 제거되었고 인덱스도 옳게 변경되었음을 확인할 수 있습니다. (상기 실행된 DataFrame들 참고)
아래 코드를 실행하면 임의의 context와 `\n`, `\\n`을 제거함으로 바뀐 인덱스에서 답변이 정상적으로 출력되는지 확인할 수 있습니다

In [32]:
import random
for i in range(10):
    x = random.randint(0,70132)
    print('context',df['context'][x])
    ans_index = df['answer_start'][x]
    print('ans_index answer' ,df['context'][x][ans_index:ans_index+5])
    print('groundtruth',df['text'][x], end = '\n\n')

context 2014년 1월 현대엘리베이터 베트남 법인이 설립되었으며, 4월에는 브라질 공장이 준공되었다. 같은 해 6월에는 대한민국 내 상용 승강기 중 최고 속도인 분속 600m로 운행하는 승강기를 부산국제금융센터에 설치하였으며, 12월에는 대한민국 내 에스컬레이터중 최장 거리인 57m를 운행하는 에스컬레이터를 대구 도시철도의 신남역 환승통로에 설치하였다. 2015년 2월에는 대한민국 최초의 더블데크 엘리베이터를 용산 LG유플러스 사옥에 설치하였으며 4월에는 승강기 유상관리대수 10만 대를 돌파하였다. 2016년 7월 이천공장 월 생산량 2,000대를 돌파하였으며 9월에는 터키 합작법인을 설립하였다. 2017년 6월에는 월간 엘리베이터 설치 대수 2,000대를 돌파하였다.
ans_index answer 2014년
groundtruth 2014년

context 1898년 10월부터 이규완은 황철, 강성형, 윤세용 등 박영효 추종자들과 일본에서 몰래 귀국하여 한성 진고개에 잠복, 일본 공사관측이 제공하는 자금으로써 친위대 소속 군인 150명과 자객 30여 명을 규합하여 고종을 폐위시키고 박영효를 총리대신으로 추대하려는 음모를 진행시키고 있었다. 10월말 다시 일본으로 돌아갔다가 11월 9일 다시 국내에 입국하여 독립협회를 지원하였다. 이 기간 중 이규완은 매부 강성형, 독립협회원 이승만, 이민칙(李敏穫) 그밖에 최정덕, 윤세용, 이규승, 강호선(姜浩善), 신창희(申昌熙) 등과 접촉하였다.1898년 11월 이규완은 황철, 강성형 등을 중심으로 황제 폐위와 박영효 추대 운동을 추진했다. 이규완은 이승만을 포섭하는데 성공한다. 독립협회와 만민공동회로 박해받던 이승만은 독립협회와 민권 운동을 탄압하는 조선에서는 더이상 견디지 못하겠다고 호소하였다. 이규완은 이승만을 곧 고종 폐위 운동에 포섭하는데 성공한다.이승만은 이규완을 찾아가서 타국에 의탁하는 길을 부탁하고 이곳에서는 오래 머물지 못하겠다는 뜻으로 언급하였으니 그 마음가짐을 헤아리기 어려우니 강성형과 대질할 때

가볍게 결측치를 찍어볼까요?  
**!주의!** 여기서 결측치가 나오면... 처리하고 넘어가야 합니다  
아니면 arrow로 만들 때 오류가 납니다

In [109]:
df.isnull().sum()

title           0
context         0
question        0
id              0
document_id     0
text            0
answer_start    0
dtype: int64

In [ ]:
이 셀을 실행하면 CSV로 내보낼 수 있습니다.

In [32]:
df.to_csv('remove_nl.csv',index=False)

### arrow 형태로 저장하는 방법입니다.
(contribute : 설유민)

In [19]:
from datasets import *
from copy import deepcopy

arrow의 nested가 문제였는데, 이건 역순으로 묶어야 하는 것끼리 list(dict()) 를 만들어 해결합니다.

In [20]:
answers = []
for datum in df.itertuples():
    answers.append({'text':[datum.text], 'answer_start':[datum.answer_start]})

In [21]:
nested_df = deepcopy(df)
nested_df['answers'] = answers
nested_df.drop(columns = ['text', 'answer_start'], inplace=True)

여기까지 해서 nested dataframe을 만들었으면, `Dataset.from_pandas`를 이용해 변환할 수 있습니다.  
반드시 아래 코드를 실행시켜 원하는 feature의 형태와 자료형이 맞는지 확인하세요

In [22]:
new_dataset = Dataset.from_pandas(nested_df)
new_dataset, new_dataset.features

(Dataset({
     features: ['title', 'context', 'question', 'id', 'document_id', 'answers'],
     num_rows: 70133
 }),
 {'title': Value(dtype='string', id=None),
  'context': Value(dtype='string', id=None),
  'question': Value(dtype='string', id=None),
  'id': Value(dtype='string', id=None),
  'document_id': Value(dtype='int64', id=None),
  'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
   'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}})

이제 valid set을 확인하고 arrow 로 묶어줄 차례입니다.  
valid set에는 변경사항이 없지만 묶어주긴 새로 묶어줘야 합니다.

In [44]:
ours_valid = load_from_disk("combined")["validation"]
ours_valid = ours_valid.remove_columns('__index_level_0__')
ours_valid, ours_valid.features

(Dataset({
     features: ['title', 'context', 'question', 'id', 'answers', 'document_id'],
     num_rows: 240
 }),
 {'title': Value(dtype='string', id=None),
  'context': Value(dtype='string', id=None),
  'question': Value(dtype='string', id=None),
  'id': Value(dtype='string', id=None),
  'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
   'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)},
  'document_id': Value(dtype='int64', id=None)})

마지막으로 둘을 arrow로 묶어주는 코드입니다.

In [45]:
preprocessed_arrow = DatasetDict({"train": new_dataset, "validation": ours_valid})
preprocessed_arrow, preprocessed_arrow["train"].features, preprocessed_arrow["validation"].features

(DatasetDict({
     train: Dataset({
         features: ['title', 'context', 'question', 'id', 'document_id', 'answers'],
         num_rows: 70133
     })
     validation: Dataset({
         features: ['title', 'context', 'question', 'id', 'answers', 'document_id'],
         num_rows: 240
     })
 }),
 {'title': Value(dtype='string', id=None),
  'context': Value(dtype='string', id=None),
  'question': Value(dtype='string', id=None),
  'id': Value(dtype='string', id=None),
  'document_id': Value(dtype='int64', id=None),
  'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
   'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}},
 {'title': Value(dtype='string', id=None),
  'context': Value(dtype='string', id=None),
  'question': Value(dtype='string', id=None),
  'id': Value(dtype='string', id=None),
  'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None),
   'text': Sequence(

찐막! 묶인 arrow를 저장합니다

In [47]:
preprocessed_arrow.save_to_disk("./preprocessed_combined")